In [91]:
# The purpose of this file is to show the functionality of trading_bot.py
# without having to run the file itself, as doing so will cause the program to make a trade.

In [92]:
#Run this code snippet to connect to the Binance API.
import yaml
from binance.client import Client

with open('.api_keys.yaml') as f:
    api_keys = yaml.safe_load(f)
    
client = Client(api_keys['binance_api'], api_keys['binance_secret'])

In [93]:
# This notebook runs through the process of making a trade on Binance without making a trade itself.

# Stage 1: Check if a file called 'portfolio/portfolio.yaml' exists.
# This is the current holdings of the portfolio.
# If it doesn't, one will be created so long as the user has 100 USDT free in their account.
# File is safe to run.

import os
def check_portfolio(client):
    '''Checks if portfolio exists yet - if it doesn't, one will be created so long as account has 100USDT'''
    if not os.path.isfile('portfolio_info/portfolio.yaml'):
        x = client.get_asset_balance(asset='USDT')['free']
        x = int(x[:x.find('.')]+x[x.find('.')+1:x.find('.')+3])
        if x < 100:
            raise Exception('Account does not have enough free USDT to create portfolio.')
        else:
            with open('portfolio_info/default_portfolio.yaml','r') as f:
                data = yaml.safe_load(f)
            with open('portfolio_info/portfolio.yaml','w') as f:
                yaml.dump(data,f)
                
check_portfolio(client)

In [94]:
# Stage 2: Assuming the file 'portfolio_info/portfolio.yaml' exists, the portfolio data
# is a extracted from this config file

def load_portfolio():
    '''Loads portfolio from config file'''
    with open('portfolio_info/portfolio.yaml','r') as f:
        data = yaml.safe_load(f)
    return data

portfolio = load_portfolio()

for x in portfolio:
    print(f'{x}: {portfolio[x]}')

ADAUSDT: {'amount': 17.8, 'currency': 'ADA'}
BTCUSDT: {'amount': 0.00049, 'currency': 'BTC'}
DOGEUSDT: {'amount': 139.0, 'currency': 'DOGE'}
ETHUSDT: {'amount': 0.006, 'currency': 'ETH'}


In [95]:
# Stage 3: Data from BitInfoCharts.com is scraped for a currency and stored to a variable.
# Dogecoin is used as the example in this snippet.

import requests
from datetime import datetime, timedelta
def get_most_recent_datapoint(currency):
    '''Returns inputs to be used in machine learning file'''
    url = f'https://bitinfocharts.com/comparison/{currency}-tweets.html'
    source = requests.get(url, 'html.parser').text
    a = source.find((datetime.utcnow()-timedelta(days=2)).strftime("%Y/%m/%d"))
    b = source.find((datetime.utcnow()-timedelta(days=1)).strftime("%Y/%m/%d"))
    c = source[b:].find(']')
    d = source[a:b+c]
    d = source[a:b+c].split(',')
    pc = 100*(float(d[3])/float(d[1][:-1])-1)
    return(d[2][11:-2], d[1][:-1], pc)

data = get_most_recent_datapoint('dogecoin')
print(f'yesterday\'s date: {data[0]}\ntweets posted that day: {data[1]}\npercentage change from previous day:{data[2]}%')

yesterday's date: 2022/01/16
tweets posted that day: 40266
percentage change from previous day:-31.356479411910797%


In [53]:
# Stage 4: A temporary ARFF file is created using the data from BitInfoCharts

def generate_temp_arff(data, c):
    '''Generates a hidden .arff file that is used by autoweka.jar machine learning model. Data must be in format ('%Y/%m/%d',int>=0,float>=-100.0) or error will be raised'''
    with open('.temp.arff','w') as f:
        f.write(f'@relation {c}_price\n\n@attribute date date yyyy/MM/dd\n@attribute tweets numeric\n@attribute tweet_change numeric\n@attribute price numeric\n\n@data\n')
        f.write(f'{data[0]},{data[1]},{data[2]},0')

generate_temp_arff(data, 'dogecoin')

In [96]:
# Stage 5: A prediction is made using the saved WEKA model and the temporary ARFF file


def predict_price(c):
    '''Temporary .arff file and machine learning model stored in data folder'''
    stream = os.popen(f'java -cp autoweka.jar weka.classifiers.meta.AutoWEKAClassifier -T .temp.arff -l models/{code(c).lower()}dl.model -p 0')
    x = float(stream.read().split("\n")[5].split()[2])
    return x*100 if c in ('dogecoin','cardano') else x
    # Why x*100 for dogecoin and cardano? As Dogecoin and Cardano are both low in value, and WEKA only
    # makes predictions to 2 decimal places, their values are done in cents. The data sets used to create
    # the models for Cardano and Dogecoin also did this.

prediction = predict_price('dogecoin')
#WEKA not working? Comment out the line above and uncomment the line below for some sample data.
#prediction = 51.1

In [97]:
# Stage 6: The current price of the asset is gathered from Binance

def code(currency):
    '''Returns three letter code used for each cryptocurrency (four letter in case of dogecoin)'''
    return 'DOGE' if currency == 'dogecoin' else 'ETH' if currency == 'ethereum' else 'BTC' if currency == 'bitcoin' else 'ADA'

def exchange(currency):
    '''Returns trading code for currency vs USDT'''
    return f'{code(currency)}USDT'

def get_price(client, c, trading=False):
    '''Binance API is used to get current price of a cryptocurrency'''
    x = float(client.get_symbol_ticker(symbol=exchange(c))['price'])
    return x*100 if c in ('dogecoin','cardano') and not trading else x

price = get_price(client, 'dogecoin')
print(price)

17.31


In [98]:
# Stage 7: The system checks whether the USDT value of the asset in the portfolio has not slipped
# below 10 USDT (whether held as USDT or not). Trades must be worth a minimum of 10 USDT.

def check_tradable(c):
    '''As Binance trades must be worth a minimum of 10USDT, a check is put in place to make sure the value of each sub-portfolio has not slipped below 10USDT'''
    data = load_portfolio()
    if data[exchange(c)]['currency']=='USDT':
        if data[exchange(c)]['amount'] >= 1000:
            return True
        return False
    else:
        price = get_price(client, c)
        price *= 100 if c in ('ethereum','bitcoin') else 1
        if data[exchange(c)]['amount']*price >= 1000:
            return True
        return False
    
check_tradable('dogecoin')

True

In [80]:
# Stage 8: Assuming check_tradable(c) is True, a cryptocurrency is bought if its current price is
# below the prediction price and sold if the price is expected to drop. The code that makes trades
# has been commented out and the functions are not included here: a print statement is used instead.

def trade(price, prediction, portfolio, c, client):
    '''Makes buy/sell/hodl decision and executes trades (see buy() and sell() methods)'''
    if prediction > price and portfolio[exchange(c)]['currency']=='USDT':
        #buy(portfolio, c, client)
        print(f"{c} bought")
    elif price > price and portfolio[exchange(c)]['currency']!='USDT':
        #sell(portfolio, c, client)
        print(f"{c} sold!")
    else:
        print(f"No buy/sell action. {portfolio[exchange(c)]['amount']} {c} is held.")

trade(price, prediction, portfolio, 'DOGE', client)

No buy/sell action. 17.8 DOGE is held.


In [69]:
# For viewing purposes only, here is the code for enacting buy/sell actions

# def buy(portfolio, c, client):
#     '''USE WITH CAUTION
#         Executes a buy decision and updates portfolio variable accordingly; p!=q due to trading fees and imprecision between converting between currencies'''
#     q = str((portfolio[exchange(c)]['amount']/100)/get_price(client,c,trading=True))
#     q = q[:q.find('.')+get_precision(c)+1]
#     if q[-1]=='.':
#         q=int(q[:-1])
#     else:
#         q=float(q)
#     if q>float(client.get_asset_balance(asset='USDT')['free'])/get_price(client, c, trading=True):
#         raise Exception("Insufficient funds to carry out transaction - too much USDT has been withdrawn")
#     client.order_market_buy(
#         symbol=exchange(c),
#         quantity=q
#     )
#     p = client.get_asset_balance(asset=code(c))['free']
#     p = p[:p.find('.')+get_precision(c)+1]
#     if p[-1]=='.':
#         p = p[:-1]
#     portfolio[exchange(c)]['amount'] = float(p)
#     portfolio[exchange(c)]['currency'] = code(c)

# def sell(portfolio, c, client):
#     '''USE WITH CAUTION
#         Executes sell decision and updates portfolio variable accordingly; p!=q due to trading fees and imprecision between converting between currencies'''
#     a = client.get_asset_balance(asset='USDT')['free']
#     usdt_before = int(a[:a.find('.')]+a[a.find('.')+1:a.find('.')+3])
#     quantity=client.get_asset_balance(asset=code(c))
#     client.order_market_sell(
#         symbol=exchange(c),
#         quantity=quantity
#     )
#     b = client.get_asset_balance(asset='USDT')['free']
#     usdt_after = int(b[:b.find('.')]+b[b.find('.')+1:b.find('.')+3])
#     portfolio[exchange(c)]['amount'] = usdt_after-usdt_before
#     portfolio[exchange(c)]['currency'] = 'USDT'

In [99]:
# Stages 3-8 are repeated for each cryptocurrency.

weka_functioning = True
for c in ('dogecoin','ethereum','bitcoin','cardano'):
    data = get_most_recent_datapoint(c)
    generate_temp_arff(data,c)
    if weka_functioning:
        prediction = predict_price(c)
    else:
        prediction = 13.0 if c=='dogecoin' else 45324.31 if c=='bitcoin' else 2953.31 if c=='ethereum' else 101.03
    price = get_price(client,c)
    if (check_tradable(c)):
        trade(price, prediction, portfolio, c, client)
        


No buy/sell action. 139.0 dogecoin is held.
No buy/sell action. 0.006 ethereum is held.
No buy/sell action. 0.00049 bitcoin is held.
No buy/sell action. 17.8 cardano is held.


In [100]:
# Final stage: configuration files are updated. Demo versions of 'portfolio_info/portfolio.yaml'
# and 'portfolio_info/portfolio_history.yaml' have been created so files are not overwritten.

def currency(code):
    '''Returns cryptocurrency name for each crypto code'''
    return 'dogecoin' if code =='DOGE' else 'ethereum' if code == 'ETH' else 'bitcoin' if code == 'BTC' else 'ADA'

def add_to_history(portfolio, client):
    '''Updates portfolio.yaml file and appends new information to portfolio_history.yaml'''
#     with open('portfolio_info/portfolio.yaml','w') as f:
#         yaml.dump(portfolio,f)
    with open('portfolio_info/portfolio_DEMO.yaml','w') as f:
        yaml.dump(portfolio,f)
    total_value = 0
    temp_portfolio = portfolio
    for k in temp_portfolio:
        temp_portfolio[k]['value'] = temp_portfolio[k]['amount'] if temp_portfolio[k]['currency']=='USDT' else round(temp_portfolio[k]['amount']*get_price(client,currency(temp_portfolio[k]['currency']),trading=True)*100)
        round(temp_portfolio[k]['amount']*get_price(client, currency(temp_portfolio[k]['currency']),trading=True))
        total_value+=temp_portfolio[k]['value']
    temp_portfolio['total_value'] = total_value
    with open('portfolio_info/portfolio_history.yaml','r') as f:
        data = yaml.safe_load(f)
        data[len(data)] = temp_portfolio
#     with open('portfolio_info/portfolio_history.yaml','w') as f:
#         yaml.dump(data,f)
    with open('portfolio_info/portfolio_history_DEMO.yaml','w') as f:
        yaml.dump(data,f)
    
add_to_history(portfolio,client)